In [1]:
import os
import pickle
os.getcwd()
os.chdir("../../")

import numpy as np
from matplotlib import pyplot as plt

In [2]:
from src.gpdssm.models import StandardSingleModel

In [3]:
def init_model(fold, cycle=1, learning_rate=0.001):
    model = StandardSingleModel(data_name="train", data_fold=fold, dim_hidden=[2], num_rff=50, num_particle=1000, 
                            display_step=2000, warm_start=0, learning_rate=learning_rate, num_train_cycle=cycle)
    return model

# def zero_round_learn(model_list):
#     for i in range(len(model_list)):
#         print("Class " + str(i) + " starts learning")
#         model_list[i].learn()

# def first_round_learn(model_list):
#     C = len(model_list)
#     new_model_list = [init_model(i, cycle_array[i-1]) for i in range(1, C+1)]
#     print("Class 0 starts learning")
#     new_model_list[0].initialize_structure_from_model(model_from_same_class = model_list[0])
#     new_model_list[0].learn()
    
#     for i in range(1, C):
#         print("Class " + str(i) + " starts learning")
#         new_model_list[i].initialize_structure_from_model(new_model_list[i-1], model_list[i])
#         new_model_list[i].learn()
        
#     return new_model_list

def first_round_learn(model_list):
    C = len(model_list)
    new_model_list = [init_model(i) for i in range(1, C+1)]

    print("Class 0 starts learning")
    new_model_list[0].learn()
    
    for i in range(1, C):
        print("Class " + str(i) + " starts learning")
        new_model_list[i].initialize_structure_from_model(new_model_list[i-1])
        new_model_list[i].learn()
        
    return new_model_list
        
def next_round_learn(model_list, cycle_array, rate_array, stop, flag):
    C = len(model_list)
    new_model_list = [init_model(i, cycle_array[i-1], rate_array[i-1]) for i in range(1, C+1)]
    
    # if not stop[0]:
    #     print("Class 0 starts learning")
    #     new_model_list[0].initialize_structure_from_model(model_list[flag], model_list[0])
    #     new_model_list[0].relearn()
    #     flag = 0
    # else:
    #     new_model_list[0] = model_list[0]
    
    for i in range(C):
        if not stop[i]:
            print("Class " + str(i) + " starts learning")
            if flag < i:
                new_model_list[i].initialize_structure_from_model(new_model_list[flag], model_list[i])
            else:
                new_model_list[i].initialize_structure_from_model(model_list[flag], model_list[i])
            new_model_list[i].relearn()
            flag = i
        else:
            new_model_list[i] = model_list[i]
        
            
    return new_model_list, flag

def get_mse_list(model_list):
    return [model_list[i].get_mse()[-1] for i in range(len(model_list))]

def check_stop(mse_list, model_list):
    new_mse_list = get_mse_list(model_list)
    ratio = np.array(new_mse_list) / np.array(mse_list)
    return new_mse_list, (ratio > 0.98) & (ratio < 1.02)

In [4]:
C = 3
cycle = [1, 1, 1]
rate = [0.001, 0.1, 0.01]
stop = [0, 0, 0]
flag = C - 1

In [5]:
model_list = [init_model(i, cycle[i-1], rate[i-1]) for i in range(1, C+1)]

# zero_round_learn(model_list)
model_list = first_round_learn(model_list)

mse_list = get_mse_list(model_list)

Class 0 starts learning
>>> train cycle 1
>>> cycle=1, t=1, mse=1.6302, mnll=6.3368, time=0.0 minutes
>>> cycle=1, t=2000, mse=0.9132, mnll=-2.4718, time=0.25 minutes
>>> cycle=1, t=4000, mse=0.6093, mnll=-4.7607, time=0.5 minutes
>>> cycle=1, t=6000, mse=0.4619, mnll=-6.1354, time=0.76 minutes
>>> cycle=1, t=8000, mse=0.3778, mnll=-7.1624, time=1.02 minutes
>>> cycle=1, t=10000, mse=0.3229, mnll=-7.9904, time=1.27 minutes
Class 1 starts learning
>>> train cycle 1
>>> cycle=1, t=1, mse=6.6589, mnll=-79800.7643, time=0.0 minutes
>>> cycle=1, t=2000, mse=3.0321, mnll=-33.7385, time=0.27 minutes
>>> cycle=1, t=4000, mse=2.3348, mnll=-20.1009, time=0.52 minutes
>>> cycle=1, t=6000, mse=2.0624, mnll=-15.6848, time=0.77 minutes
>>> cycle=1, t=8000, mse=1.9074, mnll=-13.505, time=1.02 minutes
>>> cycle=1, t=10000, mse=1.8013, mnll=-12.3863, time=1.29 minutes
Class 2 starts learning
>>> train cycle 1
>>> cycle=1, t=1, mse=1.8813, mnll=-123799.5106, time=0.0 minutes
>>> cycle=1, t=2000, mse=2.5

In [6]:
while sum(stop) != 3:
    model_list, flag = next_round_learn(model_list, cycle, rate, stop, flag)
    mse_list, stop = check_stop(mse_list, model_list)

Class 0 starts learning
>>> relearn
>>> cycle=1, t=1, mse=8.1659, mnll=-220521.5023, time=0.0 minutes
>>> cycle=1, t=2000, mse=0.3737, mnll=-118.4968, time=0.24 minutes
>>> cycle=1, t=4000, mse=0.2901, mnll=-64.3125, time=0.48 minutes
>>> cycle=1, t=6000, mse=0.2514, mnll=-46.4524, time=0.73 minutes
>>> cycle=1, t=8000, mse=0.2251, mnll=-37.6433, time=0.99 minutes
>>> cycle=1, t=10000, mse=0.2059, mnll=-32.4255, time=1.24 minutes
Class 1 starts learning
>>> relearn
>>> cycle=1, t=1, mse=10.302, mnll=-323971.9828, time=0.0 minutes
>>> cycle=1, t=2000, mse=1.7331, mnll=-168.2033, time=0.25 minutes
>>> cycle=1, t=4000, mse=1.5456, mnll=-88.5956, time=0.5 minutes
>>> cycle=1, t=6000, mse=1.5028, mnll=-62.0784, time=0.77 minutes
>>> cycle=1, t=8000, mse=1.4622, mnll=-48.7826, time=1.02 minutes
>>> cycle=1, t=10000, mse=1.4341, mnll=-41.0235, time=1.28 minutes
Class 2 starts learning
>>> relearn
>>> cycle=1, t=1, mse=15.4238, mnll=-410226.5228, time=0.0 minutes
>>> cycle=1, t=2000, mse=0.864

In [7]:
cwd = os.getcwd()
model_path = os.path.join(cwd, "models", "train.pickle")
with open(model_path, 'wb') as f:
    pickle.dump(model_list, f)